In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('src/')

from data_util import hz_to_audio, df_to_hz, pow_to_db

import matplotlib.pyplot as plt
from scipy.signal import welch
import IPython.display as ipd
from glob import glob as glob
import sounddevice as sd
import soundfile as sf
import pandas as pd
import numpy as np
import librosa
import os

In [ ]:
paths = glob("data/musicnet_solo/*.wav")
hop_size = 441

for path in paths[3:4]:
    dirname = os.path.dirname(path)
    basename = os.path.splitext(os.path.basename(path))[0]

    data, sr = sf.read(path, dtype='float32')
    f0_y, voiced_flag, voiced_probs = librosa.pyin(data, sr=sr, frame_length=hop_size * 4, fmin=50, fmax=2000)

    df = pd.read_csv(os.path.join(dirname, basename + '.csv'))

    f0_x = df_to_hz(df, new_sr=(sr / hop_size))

    n_samples = np.min([len(f0_y), len(f0_x)])
    f0_x = f0_x[:n_samples]
    f0_y = f0_y[:n_samples]

    print(n_samples)

    f0_y[np.isnan(f0_y)] = f0_x[np.isnan(f0_y)]

    f0_x[np.isnan(f0_x)] = 0
    f0_y[np.isnan(f0_y)] = 0

    # f, p_x = welch(f0_x, fs=sr)
    # f, p_y = welch(f0_y, fs=sr)

    # plt.figure()
    # plt.plot(f, pow_to_db(p_x))
    # plt.plot(f, pow_to_db(p_y))
    # plt.xlabel('Frequency')
    # plt.ylabel('Level (dB)')
    # plt.xscale('log')

    audio_x = hz_to_audio(f0_x, sr=sr, hop_size=hop_size)
    audio_y = hz_to_audio(f0_y, sr=sr, hop_size=hop_size)

    display(ipd.Audio(audio_x, rate=sr))
    display(ipd.Audio(audio_y, rate=sr))

    plt.plot(f0_x)
    plt.plot(f0_y)
    plt.show()
    break
